In [1]:
!which python3

/Users/aleksandr/Desktop/micro_llm/venv/bin/python3


In [6]:
# !pip install torch
# !pip install torchvision
# !pip install tensorboard

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

### загрузка данных

In [12]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [16]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

### создаем сетки

In [33]:
class TwoLayerNN(nn.Module):
    def __init__(self):
        super(TwoLayerNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class ThreeLayerNN(nn.Module):
    def __init__(self):
        super(ThreeLayerNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [45]:
def train(model, device, train_loader, optimizer, epoch, writer, model_name):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}')
            # Запись значения функции потерь в TensorBoard
            writer.add_scalar(f'{model_name}/train_loss', loss.item(), epoch * len(train_loader) + batch_idx)

def test(model, device, test_loader, writer, model_name, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

            if batch_idx == 0:
                img_grid = data[:10]
                class_preds = pred[:10].squeeze().cpu().numpy()
                writer.add_images(f'{model_name}/images', img_grid, epoch)
                writer.add_text(f'{model_name}/predictions', str(class_preds), epoch)

    test_loss /= len(test_loader.dataset)
    accuracy = 100 * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')
    
    # Запись значения функции потерь и accuracy в TensorBoard
    writer.add_scalar(f'{model_name}/test_loss', test_loss, epoch)
    writer.add_scalar(f'{model_name}/test_accuracy', accuracy, epoch)
        
        

### Настройка TensorBoard

In [46]:
writer1 = SummaryWriter(log_dir='runs/TwoLayerNN')
writer2 = SummaryWriter(log_dir='runs/ThreeLayerNN')

### обучение

In [47]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = torch.device(device)

model1 = TwoLayerNN().to(device)
optimizer1 = optim.SGD(model1.parameters(), lr=0.01, momentum=0.9)

model2 = ThreeLayerNN().to(device)
optimizer2 = optim.SGD(model2.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 11):
    train(model2, device, train_loader, optimizer2, epoch, writer2, 'ThreeLayerNN')
    test(model2, device, test_loader, writer2, 'ThreeLayerNN', epoch)


Train Epoch: 1 [0/60000] Loss: 2.323298
Train Epoch: 1 [6400/60000] Loss: 0.383506
Train Epoch: 1 [12800/60000] Loss: 0.347285
Train Epoch: 1 [19200/60000] Loss: 0.309093
Train Epoch: 1 [25600/60000] Loss: 0.326366
Train Epoch: 1 [32000/60000] Loss: 0.143080
Train Epoch: 1 [38400/60000] Loss: 0.048600
Train Epoch: 1 [44800/60000] Loss: 0.122946
Train Epoch: 1 [51200/60000] Loss: 0.257227
Train Epoch: 1 [57600/60000] Loss: 0.360983

Test set: Average loss: 0.1400, Accuracy: 9557/10000 (95.57%)

Train Epoch: 2 [0/60000] Loss: 0.146106
Train Epoch: 2 [6400/60000] Loss: 0.126080
Train Epoch: 2 [12800/60000] Loss: 0.232460
Train Epoch: 2 [19200/60000] Loss: 0.201120
Train Epoch: 2 [25600/60000] Loss: 0.081762
Train Epoch: 2 [32000/60000] Loss: 0.131019
Train Epoch: 2 [38400/60000] Loss: 0.154626
Train Epoch: 2 [44800/60000] Loss: 0.018392
Train Epoch: 2 [51200/60000] Loss: 0.124261
Train Epoch: 2 [57600/60000] Loss: 0.127369

Test set: Average loss: 0.0920, Accuracy: 9726/10000 (97.26%)

Tr

In [48]:
writer1.close()
writer2.close()

In [62]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 16*4*4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [63]:
writer_lenet = SummaryWriter(log_dir='runs/LeNet')

In [64]:
model_lenet = LeNet().to(device)
optimizer_lenet = optim.SGD(model_lenet.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 11):
    train(model_lenet, device, train_loader, optimizer_lenet, epoch, writer_lenet, 'LeNet')
    test(model_lenet, device, test_loader, writer_lenet, 'LeNet', epoch)

writer_lenet.close()

Train Epoch: 1 [0/60000] Loss: 2.306116
Train Epoch: 1 [6400/60000] Loss: 0.523926
Train Epoch: 1 [12800/60000] Loss: 0.142274
Train Epoch: 1 [19200/60000] Loss: 0.056528
Train Epoch: 1 [25600/60000] Loss: 0.124075
Train Epoch: 1 [32000/60000] Loss: 0.105055
Train Epoch: 1 [38400/60000] Loss: 0.150868
Train Epoch: 1 [44800/60000] Loss: 0.047761
Train Epoch: 1 [51200/60000] Loss: 0.098047
Train Epoch: 1 [57600/60000] Loss: 0.111494

Test set: Average loss: 0.0817, Accuracy: 9725/10000 (97.25%)

Train Epoch: 2 [0/60000] Loss: 0.075685
Train Epoch: 2 [6400/60000] Loss: 0.074274
Train Epoch: 2 [12800/60000] Loss: 0.018248
Train Epoch: 2 [19200/60000] Loss: 0.027636
Train Epoch: 2 [25600/60000] Loss: 0.033317
Train Epoch: 2 [32000/60000] Loss: 0.043517
Train Epoch: 2 [38400/60000] Loss: 0.032483
Train Epoch: 2 [44800/60000] Loss: 0.020993
Train Epoch: 2 [51200/60000] Loss: 0.023826
Train Epoch: 2 [57600/60000] Loss: 0.009067

Test set: Average loss: 0.0531, Accuracy: 9832/10000 (98.32%)

Tr

In [ ]:
!tensorboard --logdir=runs

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.13.0 at http://localhost:6006/ (Press CTRL+C to quit)
